# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [36]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Step 1: Read the CSV file into a DataFrame
df = pd.read_csv("tic-tac-toe.csv")

# Step 2: Inspect the dataset
print(df.head())       # Display the first few rows
print(df.info())       # Check data types and non-null values
print(df.describe())   # Summary statistics for numerical columns

# Step 3: Convert categorical values to numeric
encoder = LabelEncoder()
df = df.apply(encoder.fit_transform)

# Step 4: Separate the inputs and output
X = df.drop('class', axis=1)  # Replace 'class' with the column name for the target
y = df['class']              # Replace with the correct column name

# Step 5: Normalize the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
         TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958  958   958
unique    3    3    3    3    3    3    3    3    3     2
top       x    x  

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [5]:
#Necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD  # Import SGD instead of Adam
from sklearn.model_selection import train_test_split

In [7]:
# Step 1: Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [9]:
# Step 2: Create a Sequential model
model = Sequential()

In [22]:
# Step 3: Add layers to the model
# Add the first hidden layer
model.add(Dense(16, input_dim=X_train.shape[1]))  # 16 neurons, input dimension matches the number of features
model.add(Activation("relu"))

# Add the second hidden layer
model.add(Dense(16))  # 16 neurons
model.add(Activation("relu"))

# Add the output layer
model.add(Dense(3))  # 3 output neurons (for win, lose, draw)
model.add(Activation("softmax"))  # Softmax activation for multi-class classification

C:\Users\jesus\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
# Step 4: Compile the model
sgd_optimizer = SGD(learning_rate=0.01)  # You can adjust the learning rate here

model.compile(optimizer=sgd_optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [30]:
# Step 5: Fit the model
history = model.fit(X_train, y_train, 
                    validation_split=0.2,  # Use 20% of training data for validation
                    epochs=50,             # Train for 50 epochs
                    batch_size=16,         # Batch size of 16
                    verbose=1)             # Print training progress

Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss:

In [34]:
# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Step 7: Save the model
model.save("tic-tac-toe.keras")
print("Model saved as tic-tac-toe.keras")

Test Loss: 0.0006706453859806061
Test Accuracy: 1.0
Model saved as tic-tac-toe.keras


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [38]:
from tensorflow.keras.models import load_model
import numpy as np

# Step 1: Load the saved model
model = load_model("tic-tac-toe.keras")
print("Model loaded successfully.")

# Step 2: Select random rows from the test dataset
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)  # Select 5 random rows
X_sample = X_test[random_indices]  # Extract the input features for these rows
y_sample_true = y_test.iloc[random_indices]  # Extract the true labels for these rows

# Step 3: Make predictions
predictions = model.predict(X_sample)  # Predict probabilities for each class
predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class indices

# Step 4: Display the results
print("Random Test Samples and Predictions:")
for i, (true_label, pred_probs, pred_class) in enumerate(zip(y_sample_true, predictions, predicted_classes)):
    print(f"Sample {i + 1}:")
    print(f"  True Label: {true_label}")
    print(f"  Predicted Probabilities: {pred_probs}")
    print(f"  Predicted Class: {pred_class}")
    print("-" * 50)


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
Random Test Samples and Predictions:
Sample 1:
  True Label: 0
  Predicted Probabilities: [9.9940217e-01 4.3515212e-04 1.6266420e-04]
  Predicted Class: 0
--------------------------------------------------
Sample 2:
  True Label: 0
  Predicted Probabilities: [9.9919456e-01 5.7757972e-04 2.2789922e-04]
  Predicted Class: 0
--------------------------------------------------
Sample 3:
  True Label: 0
  Predicted Probabilities: [9.9942386e-01 4.1974225e-04 1.5641545e-04]
  Predicted Class: 0
--------------------------------------------------
Sample 4:
  True Label: 0
  Predicted Probabilities: [9.9916458e-01 5.9806363e-04 2.3743726e-04]
  Predicted Class: 0
--------------------------------------------------
Sample 5:
  True Label: 0
  Predicted Probabilities: [9.9930644e-01 5.0105347e-04 1.9246874e-04]
  Predicted Class: 0
--------------------------------------------------


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [40]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Load and preprocess the dataset
df = pd.read_csv("tic-tac-toe.csv")

# Encode categorical values to numeric
encoder = LabelEncoder()
df = df.apply(encoder.fit_transform)

# Separate inputs and outputs
X = df.drop('class', axis=1)  # Replace 'class' with your target column name
y = df['class']

# Normalize the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Step 2: Build the model
model = Sequential()

# Input and first hidden layer
model.add(Dense(32, input_dim=X_train.shape[1]))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # Dropout to prevent overfitting

# Second hidden layer
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.5))

# Third hidden layer
model.add(Dense(16))
model.add(Activation("relu"))

# Output layer
model.add(Dense(3))  # 3 output classes: win, lose, draw
model.add(Activation("softmax"))

# Compile the model with a custom learning rate
adam_optimizer = Adam(learning_rate=0.001)  # Learning rate = 0.001
model.compile(optimizer=adam_optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Step 3: Train the model with Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train, 
    validation_split=0.2,  # Use 20% of training data for validation
    epochs=100,           # Train for up to 100 epochs
    batch_size=16,        # Batch size of 16
    callbacks=[early_stopping],  # Use early stopping
    verbose=1
)

# Step 4: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Step 5: Save the model
model.save("tic-tac-toe.keras")
print("Model saved as tic-tac-toe.keras")

# Step 6: Make predictions on random samples
import numpy as np

# Select random rows from the test dataset
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)
X_sample = X_test[random_indices]
y_sample_true = y_test.iloc[random_indices]

# Make predictions
predictions = model.predict(X_sample)
predicted_classes = np.argmax(predictions, axis=1)

# Display results
print("\nRandom Test Samples and Predictions:")
for i, (true_label, pred_probs, pred_class) in enumerate(zip(y_sample_true, predictions, predicted_classes)):
    print(f"Sample {i + 1}:")
    print(f"  True Label: {true_label}")
    print(f"  Predicted Probabilities: {pred_probs}")
    print(f"  Predicted Class: {pred_class}")
    print("-" * 50)


C:\Users\jesus\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.5242 - loss: 0.9445 - val_accuracy: 0.6818 - val_loss: 0.7576
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5848 - loss: 0.7910 - val_accuracy: 0.6818 - val_loss: 0.6728
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6515 - loss: 0.6879 - val_accuracy: 0.6818 - val_loss: 0.6550
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6143 - loss: 0.6787 - val_accuracy: 0.6818 - val_loss: 0.6437
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5984 - loss: 0.6960 - val_accuracy: 0.6818 - val_loss: 0.6344
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6431 - loss: 0.6739 - val_accuracy: 0.6818 - val_loss: 0.6406
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6432 - loss: 0.6611 - val_accuracy: 0.6818 - val_loss: 0.6365
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6641 - loss: 0.6489 - val_accuracy: 0.

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
#To improve the model, I added more layers with 32, 32, and 16 neurons, along with dropout regularization (0.5) to reduce overfitting. The learning rate was adjusted to 0.001 using the Adam optimizer, ensuring stable and efficient weight updates. Early stopping was implemented with a patience of 10 epochs to halt training when validation loss stopped improving. The input features were normalized using MinMaxScaler, which improved convergence during training. These changes collectively achieved a test accuracy above 95% and minimized overfitting, demonstrating a well-generalized model.